In [57]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [58]:
# Create list of 2019-2020 csv file names for import
# (Note: Downloaded csv files saved outside GitHub respository due to memory restrictions)

import_files = []

for x in range(2019,2021):
    for y in range(1,13):
        if (len(str(y)) == 1):
            month = '0' + str(y)
            import_files.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
        else:
            month = str(y)
            import_files.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
            
# csv data format change begins Feb'21, so also append just Jan'21 in old format
import_files.append("../Resources/202101-citibike-tripdata.csv")
import_files.append("../Resources/JC-202101-citibike-tripdata.csv")

In [59]:
# Loop through 2019-Jan'2021 csv files and combine
df_list = (pd.read_csv(file) for file in import_files)

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)

In [60]:
# Update usertype values to reflect the 2021 dataset's better description
# Also add column for 2021 dataset's added bike-type
df['usertype'] = df['usertype'].replace(to_replace='Subscriber', value='member')
df['usertype'] = df['usertype'].replace(to_replace='Customer', value='casual')
df['rideable_type'] = 'N/A'

In [61]:
# Create list of 2021 csv file names for import (Note: Jan'21 already imported above)
# (Note: Downloaded csv files saved outside GitHub respository due to memory restrictions)
import_files_2021 = []

for x in range(2021,2022):
    for y in range(2,13):
        if (len(str(y)) == 1):
            month = '0' + str(y)
            import_files_2021.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files_2021.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")
        else:
            month = str(y)
            import_files_2021.append("../Resources/" + str(x) + month + "-citibike-tripdata.csv")
            import_files_2021.append("../Resources/JC-" + str(x) + month + "-citibike-tripdata.csv")   

In [62]:
# Loop through 2021 csv files and combine
df_list_2021 = (pd.read_csv(file) for file in import_files_2021)

# Concatenate all DataFrames
df_2021 = pd.concat(df_list_2021, ignore_index=True)

/Users/coreylawson-enos/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)
/Users/coreylawson-enos/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [63]:
# Convert start/end fields to datetime for trip duration calculation
df_2021['started_at'] = pd.to_datetime(df_2021['started_at'])
df_2021['ended_at'] = pd.to_datetime(df_2021['ended_at'])

In [64]:
# Add trip duration to dataset
df_2021['tripduration'] = (df_2021['ended_at'] - df_2021['started_at']).dt.total_seconds()

In [65]:
# Convert 2021 data to appendable dataframe
df_2021_clean = pd.DataFrame({
    'tripduration': df_2021['tripduration'],
    'starttime': df_2021['started_at'],
    'stoptime': df_2021['ended_at'],
    'start station id': df_2021['start_station_id'],
    'start station name': df_2021['start_station_name'],
    'start station latitude': df_2021['start_lat'],
    'start station longitude': df_2021['start_lng'],
    'end station id': df_2021['end_station_id'],
    'end station name': df_2021['end_station_name'],
    'end station latitude': df_2021['end_lat'],
    'end station longitude': df_2021['end_lng'],
    'bikeid': 'N/A',
    'usertype': df_2021['member_casual'],
    'birth year': 'N/A',
    'gender': 'N/A',
    'rideable_type': df_2021['rideable_type']
    })

In [66]:
# Append 2021 data to 2019-2020
df = df.append(df_2021_clean, ignore_index = True)

In [67]:
# Convert start/end times to datetime
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])

# Create yyyy-mm column for later use with visualizations
df['year_month'] = pd.to_datetime(df['starttime']).dt.to_period('M')

In [68]:
# Prepare summary data by month for export
# Group data by year-month
df_grouped = df.groupby([df['starttime'].dt.year.rename('year'), df['starttime'].dt.month_name().rename('month'), df['year_month']])

In [69]:
# Calculate year/month summary data for visualizations
total_rides = df_grouped['starttime'].count()
avg_duration = df_grouped['tripduration'].mean() / 60
total_member = df_grouped['usertype'].apply(lambda x: (x=='member').sum())
total_casual = df_grouped['usertype'].apply(lambda x: (x=='casual').sum())
total_classic = df_grouped['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
total_electric = df_grouped['rideable_type'].apply(lambda x: (x=='electric_bike').sum())

In [70]:
# Assemble year/month data in a DataFrame
time_summary = pd.DataFrame({
    'Total Rides': total_rides,
    'Average Ride (min)': avg_duration.round(0),
    'Member Rides': total_member,
    'Casual Rides': total_casual,
    'Classic Bike Rides': total_classic,
    'Electric Bike Rides': total_electric
    })

In [71]:
# Export year/month data to csv for Tableau import
time_summary.to_csv('Data/time_summary.csv')

In [15]:
# Prepare summary data by bike type for export
# Filter records for classic/electric bikes only
df_by_ride = df[(df['rideable_type'] == 'classic_bike') | (df['rideable_type'] == 'electric_bike')]

In [16]:
# Group bike records by type
df_grouped_user = df_by_ride.groupby([df['usertype']])

In [17]:
# Calculate summary figures by bike type
classic_rides = df_grouped_user['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
electric_rides = df_grouped_user['rideable_type'].apply(lambda x: (x=='electric_bike').sum())

In [18]:
# Assemble bike-type data in a DataFrame
rides_summary = pd.DataFrame({
    'Classic': classic_rides,
    'Electric': electric_rides
    })

In [19]:
# Export bike-type data to csv for Tableau import
rides_summary.to_csv('Data/rides_summary.csv')

In [20]:
# Prepare summary data by hour for export
# Group ride records by military hour (0-23)
df_grouped_hour = df.groupby([df['starttime'].dt.year.rename('year'), df['starttime'].dt.hour.rename('Hour'), df['year_month']])

In [21]:
# Calculate number of rides started for each hour
count_per_hour = df_grouped_hour['starttime'].count()

In [22]:
# Export hourly counts to csv for Tableau import
count_per_hour.to_csv('Data/hour_summary.csv')

In [23]:
# Prepare summary data by station for export
# Clean NaN 
# df = df[df['starttime'].notna()]
df = df[df['start station name'].notna()]
# df = df[df['stoptime'].notna()]
df = df[df['end station name'].notna()]
df = df[df['start station latitude'].notna()]
df = df[df['start station longitude'].notna()]
# df = df[df['end station latitude'].notna()]
# df = df[df['end station longitude'].notna()]
df = df[df['year_month'].notna()]

In [24]:
# Group data by year-month, and start/end station names
df_grouped_station = df.groupby([df['year_month'],
                                 df['start station name'].rename('station name')])

df_grouped_ends = df.groupby([df['year_month'],
                              df['end station name'].rename('station name')])

In [25]:
# Calculate number of rides started/ended at each station
starts_by_station = df_grouped_station['start station name'].count().rename('start counts')
ends_by_station = df_grouped_ends['end station name'].count().rename('end counts')

In [26]:
starts_by_station

year_month  station name                
2019-01     1 Ave & E 110 St                 776
            1 Ave & E 16 St                 4236
            1 Ave & E 18 St                 3261
            1 Ave & E 30 St                 2461
            1 Ave & E 44 St                 2110
                                            ... 
2021-12     Wyckoff Av & Stanhope St         926
            Wyckoff Ave & Gates Ave          965
            Wyckoff St & Nevins St           601
            Wythe Ave & Metropolitan Ave    3001
            Yankee Ferry Terminal            184
Name: start counts, Length: 40860, dtype: int64

In [27]:
ends_by_station

year_month  station name                
2019-01     1 Ave & E 110 St                 787
            1 Ave & E 16 St                 4328
            1 Ave & E 18 St                 3250
            1 Ave & E 30 St                 2578
            1 Ave & E 44 St                 2118
                                            ... 
2021-12     Wyckoff Av & Stanhope St         935
            Wyckoff Ave & Gates Ave          931
            Wyckoff St & Nevins St           605
            Wythe Ave & Metropolitan Ave    3140
            Yankee Ferry Terminal            201
Name: end counts, Length: 40977, dtype: int64

In [28]:
starts_df = pd.DataFrame(starts_by_station)
ends_df = pd.DataFrame(ends_by_station)

In [29]:
# Merge start/end counts into one dataset on year-month-station
station_data = pd.merge(starts_df, ends_df, how="outer", on=["year_month", "station name"])

In [30]:
# Create DataFrame, and convert any NaN created by outer merge to zero
station_data_df = pd.DataFrame(station_data)
station_data_df = station_data_df.fillna(0)

# Calculate total starts/stops for each station
station_data_df['Total Starts/Stops'] = station_data_df['start counts'] + station_data_df['end counts']
station_data_df

start counts  end counts  \
year_month station name                                         
2019-01    1 Ave & E 110 St                 776.0       787.0   
           1 Ave & E 16 St                 4236.0      4328.0   
           1 Ave & E 18 St                 3261.0      3250.0   
           1 Ave & E 30 St                 2461.0      2578.0   
           1 Ave & E 44 St                 2110.0      2118.0   
...                                           ...         ...   
2021-12    Morgan Bike Mechanics              0.0         5.0   
           Morgan Loading Docks               0.0         1.0   
           Nassau St\t& Duffield St           0.0         5.0   
           Pier 40 Dock Station               0.0        66.0   
           W 40 St & 5 Ave                    0.0         1.0   

                                     Total Starts/Stops  
year_month station name                                  
2019-01    1 Ave & E 110 St                      1563.0  
           1 Ave & E 16 St                       8564.0  
           1 Ave & E 18 St                       6511.0  
           1 Ave & E 30 St                       5039.0  
           1 Ave & E 44 St                       4228.0  
...                                                 ...  
2021-12    Morgan Bike Mechanics                    5.0  
           Morgan Loading Docks                     1.0  
           Nassau St\t& Duffield St                 5.0  
           Pier 40 Dock Station                    66.0  
           W 40 St & 5 Ave                          1.0  

[40978 rows x 3 columns]

In [41]:
# Export summary station data to csv for Tableau import
station_data_df.to_csv('Data/station_data.csv')

In [42]:
station_coord = df.filter(['start station name', 'start station latitude', 'start station longitude'], axis=1).copy()
station_coord = station_coord.sort_values("start station name")
station_coord = station_coord.dropna(how='any')
station_coord_final = station_coord.drop_duplicates(subset ="start station name", keep = 'first')

In [43]:
# Export summary station data to csv for Tableau import
station_coord_final.to_csv('Data/station_coord.csv')